In [ ]:
! pip install imbalanced-learn

In [2]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

In [3]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
file = tf.keras.utils
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
raw_df.head()

In [ ]:
raw_df[['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V26', 'V27', 'V28', 'Amount', 'Class']].describe()

In [ ]:
data=raw_df
#Rename Class
data.rename(columns={"Class": "isFraud"}, inplace=True)

#Percentage of fraud
fraud_per = data[data.isFraud == 1].isFraud.count() / data.isFraud.count()
print(fraud_per)

In [ ]:
# Looking for missing data
print(data.isnull().any().sum())

In [ ]:
#Correlation Plot
plt.figure(figsize = (14,10))
plt.title('Correlation Plot', size = 20)
corr = data.corr()
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidths=.1,cmap="Blues",fmt='.1f',annot=True)
plt.show()

In [9]:
# Defining x and y
y = data["isFraud"]
x = data.drop(["isFraud"], axis = 1)

In [10]:
#Standardization
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [11]:
# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
# SMOTE
X_train_SMOTE, y_train_SMOTE = SMOTE().fit_resample(X_train, y_train)
#SMOTE plot
pd.Series(y_train_SMOTE).value_counts().plot(kind="bar")
plt.title("Balanced Dataset")
plt.show()

In [ ]:
# DNN
layers = keras.layers
model = keras.Sequential([
layers.Dense(input_dim = 30, units = 128, activation = "relu"),
layers.Dense(units= 64, activation = "relu"),
layers.Dropout(0.2),
layers.Dense(units= 32, activation = "relu"),
layers.Dropout(0.2),
layers.Dense(units= 32, activation = "relu"),
layers.Dropout(0.2),
layers.Dense(units= 16, activation = "relu"),
layers.Dropout(0.2),
layers.Dense(units=1, activation = "sigmoid")])
model.summary()

In [ ]:
# Metrics
metrics = [
    keras.metrics.Accuracy(name="Accuracy"),
    keras.metrics.Precision(name="Precision"),
    keras.metrics.Recall(name="Recall")]
# Compiling and fiting the model
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = metrics)
model.fit(X_train_SMOTE, y_train_SMOTE, batch_size = 32, epochs = 100)
print("Evaluate on test data")
score = model.evaluate(X_test, y_test)
print("test loss, test accuracy, test precision, test recall:", score)

In [ ]:
model.summary()

In [ ]:
model.save('saved_model/epoch_100')

In [ ]:
! pip install openvino-dev[tensorflow2,onnx]

In [ ]:
! mo --saved_model_dir saved_model/epoch_100

In [ ]:
# Below this cell and subsequent cells are activities related to prediction
model = tf.keras.models.load_model('saved_model/epoch_100')

In [ ]:
# ## YOU MUST PASTE YOUR OWN ROUTE INFORMATION HERE
my_route = 'http://model-server-sample-ovms-sughosh-dev.apps.sandbox-m2.ll9k.p1.openshiftapps.com'
sub_path = '/v1/models/smote/versions/1'
# ## YOU MUST PASTE YOUR OWN ROUTE INFORMATION HERE


import requests, json
my_req = '{"instances": [[-0.81527562, -0.62780094,  1.18457726, -0.56138278,  1.97545981, -1.38669424, -0.03372776, -1.08378356, -0.46514641, -1.07813139, -2.98031409,  2.29087639, -2.82230106, 0.76695155, -5.65368683, 0.04526619, -4.77118557, -5.04520325, -3.02616084,  1.14274513, 0.35082495,  1.64467922,  0.38254332,  0.03085198,  0.83964697, -0.38594229, -0.51760032,  1.39294962,  0.22815041, 0.3301235]]}'
# my_req = '{"instances": [[1.31445961, -0.17854688, 0.56477646, 0.09430623, -0.46407724, 0.84751917, -0.55046642, 0.81641881, -0.05950402, -0.27496291, -1.09510609, -0.83743209, -0.62095535, -0.81030155, -0.80413334, -0.11816435, 0.29553654, 0.57938864, -0.1771897, -0.18193074, -0.12517763, -0.36967874, -1.14814439, -0.04861747,0.80911052, -0.77658417, 0.27860292, 0.19034682, 0.53187267, -0.34531318]]}'
# X_test.tolist()
# my_req = '{' + json.dumps(X_test.tolist()) + '}'
# my_req
response = requests.post(my_route + sub_path +':predict', my_req)
response.json()

In [ ]:
model.summary()

In [ ]:
X_test[]

In [ ]:
#Converting X_test array to numpy array before predicting
pred = model.predict(np.array(X_test))
pred = np.where(pred > 0.5, 1, 0)
pred

In [ ]:
# Checking accuracy of the prediction by the model.
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

In [ ]:
# Because it was an imbalanced dataset and the SMOTE technique was applied to training set and
# not testing set therefore the above accuracy is not correct but the f1 score below gives a better accuracy.
from sklearn.metrics import f1_score
f1_score(y_test, pred, zero_division=1)

In [ ]:
# The cofusionmatrix and the plotting shows how many wrong prediction our model did because it is not perfect. 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['non fraud', 'fraud'])
disp.plot()
plt.show()

In [ ]:
# the below code tell me which indexes in y_test are fraud. consequently those indicies in X_test 
# can be feed to the model to check if it predicting correctly. Chances are high it will because 
# it from the same dataset using which it was trained.
np.where(y_test == 1)